In [1]:
from libqubit2d import *
from libqubitNd import *


In [72]:
class Simple2dPropagator(Model):
    default = Empty()
    default.params = {
        'a' : 1,
        'wxx' : 0.05,
        'f' : 1,
        'p' : 0
    }
    default.Hs = [ tensor(sigmax(), sigmax()), tensor(sigmax(), sigmay()), tensor(sigmaz(), sigmax()), tensor(sigmaz(), sigmay())] 
    default.shapes = [lambda p : p['wxx']/4/(1+p['a']**2/p['f']**2)*np.cos(p['p']),
                      lambda p : p['wxx']/4/(1+p['a']**2/p['f']**2)*np.sin(p['p']),
                      lambda p : p['wxx']/4/(1+p['a']**2/p['f']**2)*np.cos(p['p'])*p['a']/p['f'],
                      lambda p : p['wxx']/4/(1+p['a']**2/p['f']**2)*np.sin(p['p'])*p['a']/p['f']
                     ]
                      
    #    'wxx/4/(1+a**2/f**2)*cos(p)','wxx/4/(1+a**2/f**2)*sin(p)','wxx/4/(1+a**2/f**2)*cos(p)*a/f','wxx/4/(1+a**2/f**2)*sin(p)*a/f']
    default.timeslots = {
        'slots' : 5,
        'eachtime' : 20
    }
    default.var_params = ['f','p']
    class_params = ['Hs','shapes','timeslots','var_params','params']
    def getH(self):
        return sum([H*f(self.params) for H,f in zip(self.Hs, self.shapes)])
    def update_or_copy(self, kvargs, params, just_update = False):
        super().update_or_copy( kvargs, params, just_update)
        self.H = self.getH()
        self.basis_like()
    def timelist(self,starttime = 0):
        return np.linspace(starttime, starttime + self.timeslots['eachtime'],2)
    def count_once(self,states_list = []):
        def foo(initial_state, parameters):
            self.params.update(parameters)
            #print(self.pulse.time)
            #print(self.pulse.params)
            nonlocal states_list
            slot_index = len(states_list)
            result = mesolve(self.getH(),initial_state, self.timelist(slot_index*self.timeslots['eachtime']), [],[], progress_bar=None)
            states_list += result.states[1:]
            return result.states[-1]
        return foo
    
    def basis_like(self):
        op = self.Hs[0]
        dim = op.shape[0]
        self.basis = []
        for i in range(dim):
            ket = basis(dim,dim-i-1)
            ket.dims = [op.dims[0],[1]]
            self.basis.append(ket)
            #print(self.basis)
        
    
    def count_gate(self):
        def count_gate_anon(parameters_list):    
            psi = self.basis
            result = []
            states_lists = []
            
            parameters_list_of_small_dicts = []
            
            for slot in range(self.timeslots['slots']):
                small_dict = {}
                for par_id,par in enumerate(self.var_params):
                    small_dict[par] = parameters_list[slot + par_id*self.timeslots['slots']]
                parameters_list_of_small_dicts.append(small_dict)
            #print(parameters_list_of_small_dicts)
            for psi_id, psi0 in enumerate(psi):
                states_list = [psi0]
                state_final = reduce(self.count_once(states_list),parameters_list_of_small_dicts,psi0) 
                res_state = []
                #print(state_final)
                for psi1 in psi:
                    res_state.append((psi1.dag()*state_final).data.toarray()[0][0])
                result.append(res_state)
                states_lists.append(states_list)
            return Qobj(result).trans(),states_lists
        return count_gate_anon
    def count_fidelity(self, parameters, ideal_gate):
        return self.pars_to_infidelity(ideal_gate)(parameters)
    def pars_to_infidelity(self, ideal_gate):
        def pars_to_infidelity_f(parameters):
            gate = self.count_gate()(parameters)[0]
            gate = gate / gate.data.toarray()[0][0]
            gate.dims = ideal_gate.dims
            res1 = abs(1 - fidelity(gate,ideal_gate)/ideal_gate.shape[0])
            res2 = abs(1 - abs((gate.dag()*ideal_gate).tr())/ideal_gate.shape[0])
            res = max(res1,res2)
            try:
                cur_best = self.res
            except AttributeError:
                cur_best = 1
            if cur_best > res:
                self.res = res
                self.last_parameters = parameters
            #print(self.res, res)
            return res
        return pars_to_infidelity_f
    
    def grape(self, ideal_gate, initial_guess, method='BFGS', tol = 1e-8, opts = {}):
        """
            ideal_gate : QObj 
            initial_guess : dictionary as {
                'par1' : [par1(t1), par1(t2), par1(t3), ...]
            }
        """
        self.res = 1
        return minimize(self.pars_to_infidelity(ideal_gate), initial_guess, method = method, tol = tol, options = opts)
    
    def global_grape(self, ideal_gate, initial_guess, minimizer_kwargs = None):
        self.res = 1
        return basinhopping(self.pars_to_infidelity(ideal_gate),initial_guess,minimizer_kwargs=minimizer_kwargs)

In [73]:
spopt = Simple2dPropagator()

In [74]:
f = np.pi*spopt.params['a']/spopt.params['wxx']/(spopt.timeslots['eachtime']*spopt.timeslots['slots'])


In [75]:
initial_guess_f = [f for i in range(spopt.timeslots['slots'])] + [0 for i in range(spopt.timeslots['slots'])]

In [76]:
initial_guess_f

[0.6283185307179586,
 0.6283185307179586,
 0.6283185307179586,
 0.6283185307179586,
 0.6283185307179586,
 0,
 0,
 0,
 0,
 0]

In [77]:
op,states = spopt.count_gate()(initial_guess_f)

In [78]:
op.dims = [[2,2],[2,2]]

In [79]:
op

Quantum object: dims = [[2, 2], [2, 2]], shape = [4, 4], type = oper, isherm = False
Qobj data =
[[ 0.78690319+0.j          0.00000000+0.52249902j  0.00000000+0.j
   0.00000000-0.32829582j]
 [ 0.00000000+0.52249902j  0.78690319+0.j          0.00000000-0.32829582j
   0.00000000+0.j        ]
 [ 0.00000000+0.j          0.00000000-0.32829582j  0.78690319+0.j
   0.00000000-0.52249902j]
 [ 0.00000000-0.32829582j  0.00000000+0.j          0.00000000-0.52249902j
   0.78690319+0.j        ]]

In [80]:
ZX = (+0.5*1j*1/2*np.pi*tensor(sigmaz(),sigmax())).expm()

In [81]:
ZX

Quantum object: dims = [[2, 2], [2, 2]], shape = [4, 4], type = oper, isherm = False
Qobj data =
[[ 0.70710678+0.j          0.00000000+0.70710678j  0.00000000+0.j
   0.00000000+0.j        ]
 [ 0.00000000+0.70710678j  0.70710678+0.j          0.00000000+0.j
   0.00000000+0.j        ]
 [ 0.00000000+0.j          0.00000000+0.j          0.70710678+0.j
   0.00000000-0.70710678j]
 [ 0.00000000+0.j          0.00000000+0.j          0.00000000-0.70710678j
   0.70710678+0.j        ]]

In [82]:
fidelity(op,ZX)

3.081508693993758

In [83]:
in_f = initial_guess_f
for i in range(1000):
    spopt.count_fidelity(generate_random_f(in_f[0:5]) + in_f[5:], ZX)

In [87]:
pars = spopt.grape(ZX,initial_guess_f,tol=1e-3, opts = {'maxiter' : 2,'gtol': 3e-3})

KeyboardInterrupt: 

In [86]:
initial_guess_f = spopt.last_parameters


In [ ]:
op,states = spopt.count_gate()(spopt.last_parameters)

In [ ]:

op = op* op0.conj() / abs(op0)**2

In [88]:
spopt.res

0.10092024388278387

In [ ]:
abs(1 - fidelity(op,ZX)/ZX.shape[0])

In [ ]:
op0 = op.data.toarray()[0][0]

In [ ]:
op0

In [ ]:
op0 * op0.conj() / abs(op0)**2

In [ ]:
in_f = [1,1,1,1,1,0,0,0,0]
generate_random_f(in_f[0:5]) + in_f[5:]